# **Simulations performed using RiboNpy**

# Import related pacakages

In [ ]:
import cobra
import sys
import pandas as pd
import numpy as np
import plotly.express as px

sys.path.append(r'./code/')
from ribonpy_core_functions import *

# All-in-one enzyme-constraint function

In [ ]:
model_name = './data/models/toy_model.xml' 
obj_reaction = 'DM_e4p_c'

sequence_length = 79
nucleotide_proportion = [0.25,0.25,0.25,0.25]
random_proportion = False
new_ribozyme = True

reaction_fraction = 0
selection_type = 'random'
new_reaction_selection = True

biosynth_costs = False
biosynth_costs_penalty = 1.320773

ef_factor = 1

obj_solution = get_ribozyme_constraint_model_pfba_obj_solution(model_name, obj_reaction, sequence_length, nucleotide_proportion, random_proportion, new_ribozyme, reaction_fraction, selection_type, new_reaction_selection, biosynth_costs, biosynth_costs_penalty, ef_factor)

# Effect of ribozyme sequences, structures and weights

In [ ]:
model_name = './data/models/toy_model.xml' 
obj_reaction = 'DM_e4p_c'

sequence_lengths = [35, 79, 115, 377, 754, 893, 1131]
nucleotide_proportions = [[0.25, 0.25, 0.25, 0.25],
                          [0.10, 0.40, 0.40, 0.10],
                          [0.40, 0.10, 0.10, 0.40]] #AGCU

results = []

for sequence_length in sequence_lengths:
    for nucleotide_proportion in nucleotide_proportions:
        random_proportion = False
        new_ribozyme = True
        reaction_fraction = 1
        selection_type = 'random'
        new_reaction_selection = True
        biosynth_costs = True
        biosynth_costs_penalty = 1.320773
        ef_factor = 0.1
            
        obj_solution = get_ribozyme_constraint_model_pfba_obj_solution(model_name, obj_reaction, sequence_length, nucleotide_proportion, random_proportion, new_ribozyme, reaction_fraction, selection_type, new_reaction_selection, biosynth_costs, biosynth_costs_penalty, ef_factor)
        results.append([sequence_length, nucleotide_proportion, obj_solution])

sequence_proportions_results_df = pd.DataFrame(results, columns=['Sequence Length', 'Nucleotide Proportions', 'Flux'])

# save the results to a CSV file
sequence_proportions_results_df.to_csv('analysis/simulations/RiboNpy_ribozyme_structure_variations_ef01.csv', index=False)
sequence_proportions_results_df

In [ ]:
#Convert the 'Nucleotide Proportions' list column to a string column
sequence_proportions_results_df['Nucleotide Proportions'] = sequence_proportions_results_df['Nucleotide Proportions'].apply(lambda x: ','.join(map(str, x)))

fig = px.bar(sequence_proportions_results_df, x='Sequence Length', y='Flux', color='Nucleotide Proportions', barmode='group', hover_data=['Flux'])
fig.show()

# Effects of ribozyme activity levels

In [ ]:
model_name = './data/models/toy_model.xml' 
obj_reaction = 'DM_e4p_c'

sequence_lengths = [79, 115, 377, 754]
ef_factors = [1, 0.5, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005, 0.00001]

results = []

for sequence_length in sequence_lengths:
    for ef_factor in ef_factors:
        nucleotide_proportion = [0.25, 0.25, 0.25, 0.25]
        random_proportion = False
        new_ribozyme = True
        reaction_fraction = 1
        selection_type = 'random'
        new_reaction_selection = True
        biosynth_costs = True
        biosynth_costs_penalty = 1.320773
        
        obj_solution = get_ribozyme_constraint_model_pfba_obj_solution(model_name, obj_reaction, sequence_length, nucleotide_proportion, random_proportion, new_ribozyme, reaction_fraction, selection_type, new_reaction_selection, biosynth_costs, biosynth_costs_penalty, ef_factor)
        
        modified_reaction_kcat_mw_df = calculate_modified_reaction_kcat_mw(reaction_kcat_file, modified_biosynthesis_reaction_MW_file, modified_reaction_kcat_MW_file, select_key, ef_factor)

        kcat_MW_change = round(modified_reaction_kcat_mw_df['kcat_MW'].mean(),2)

        results.append([sequence_length, ef_factor, kcat_MW_change, obj_solution])

efficiency_results_df = pd.DataFrame(results, columns=['Sequence Length', 'Efficiency factor', 'kcat_MW', 'Flux'])

# save the results to a CSV file
efficiency_results_df.to_csv('analysis/simulations/RiboNpy_ribozyme_efficiency_variations.csv', index=False)
efficiency_results_df

In [ ]:
fig = px.bar(efficiency_results_df, x='Efficiency factor', y='Flux', color='Sequence Length', barmode='group', hover_data=['Flux'])
fig.show()